In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Multi genbank and stacking



In [ ]:
#Using the example E. coli genome data from the package
import genomenotebook as gn
from genomenotebook.utils import get_cds_name, add_z_order, parse_genbank
import os
import pandas as pd
from Bio import SeqIO
from bokeh.io import output_notebook #|hide_line
from bokeh.plotting import show as bk_show #|hide_line
from bokeh.layouts import column, row #|hide_line
from bokeh.plotting import output_file, save #|hide_line
from collections import OrderedDict

/home/sean/miniconda3/envs/genomenotebook/lib/python3.9/site-packages/Bio/__init__.py:138: BiopythonWarning: You may be importing Biopython from inside the source tree. This is bad practice and might lead to downstream issues. In particular, you might encounter ImportErrors due to missing compiled C extensions. We recommend that you try running your code from outside the source tree. If you are outside the source tree then you have a setup.py file in an unexpected directory: /home/sean/miniconda3/envs/genomenotebook/lib/python3.9/site-packages
  warnings.warn(


In [ ]:

output_notebook(hide_banner=True) #|hide_line
data_path = gn.get_example_data_dir()
gb_path = os.path.join(data_path, "colored_genbank.gb")
g = gn.GenomeStack.from_genbank(gb_path,
                                width=1000, 
                                show_seq=True, 
                                search=False, 
                                feature_types=["CDS", "Domainator"], 
                                color_attribute="Color", 
                                label_angle=0, 
                                show_labels=True, 
                                feature_height=0.15, 
                                label_vertical_offset=-0.15, 
                                label_justify="left", 
                                label_horizontal_offset = 5, 
                                glyphs=gn.get_default_glyphs(box_colors=("Cyan",)), 
                                feature_name={"CDS":"gene_id","Domainator":"name"},
                                z_stack=True) #, "Domainator", "Domain_Search"
# for browser in g.browsers:
#     print(browser.show_seq)
g.show()
g.save_html("test2.html", title="stacked_contigs")

In [ ]:
g.save("test3.svg")

In [ ]:
gff_path = os.path.join(data_path, "colored_genbank.gff")
#gb_path = os.path.join(data_path, "colored_genbank.gb")
gb = gn.GenomeBrowser(gff_path = gff_path,
                       color_attribute="Color", 
                       label_angle=0, 
                       show_labels=True, 
                     )
gb.attributes
#gb.show()

In [ ]:
gb.show()